In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/../resort/rasotools-master/')
import rasotools
np.seterr(all='warn')
np.errstate(divide='raise')

## Calculating the estimate time for calculation:

In [2]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/*.nc')
files[6020]

'/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-54102_CEUAS_merged_v0.nc'

In [3]:
b = os.path.getsize(files[6020])
b

122974841

In [4]:
t = 2633.3687977790833/60
t

43.88947996298472

In [5]:
b/t

2801920.667634109

Speed: ~2.8 MB/(min thread)

In [6]:
dirsize = 0 
for i in files:
    dirsize += os.path.getsize(i)
dirsize

618842993516

In [7]:
mintime = (dirsize)/(b/t)

In [8]:
mintime/60/24 , 'days'

(153.3776754619646, 'days')

~ 1 week on 20 cores -> more cores available on the jet?

---
---

In [9]:
filesout = glob.glob('/raid60/scratch/uli/converted/*')
filesout

['/raid60/scratch/uli/converted/0-20000-0-22152_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11019_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-54102_CEUAS_merged_v0.nc',
 '/raid60/scratch/uli/converted/0-20000-0-11035_CEUAS_merged_v0.nc']

In [10]:
# with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
with eua.CDMDataset(filesout[-1]) as test:

    print(test)
    print(test.recordindices)
    print(test.observations_table)
    print(test.era5fb)
    print(test.era5fb['an_depar@body'][:])
    print(test.era5fb['fg_depar@body'][:])
    a = test.to_dataframe(groups=['observations_table', 'era5fb'], variables=['z_coordinate', 'date_time','observed_variable', 'observation_value', 'conversion_flag', 'conversion_method', 'fg_depar@body', 'an_depar@body','biascorr@body', 'biascorr_fg@body'])
a

File: <HDF5 file "0-20000-0-11035_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/converted/0-20000-0-11035_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (26625,)
 - G | era5fb_______________________________________ : : 71
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 18
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type____________________________ : : 

,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,1990.750000,1951-12-01 03:00:00,0,100000.000000,-2147483648,-2147483648,52.816605,84.320702,0.0,0.0
1,1990.750000,1951-12-04 03:00:00,0,100000.000000,-2147483648,-2147483648,-16.243063,-19.830339,0.0,0.0
2,1961.329956,1951-12-14 03:00:00,0,100000.000000,-2147483648,-2147483648,-87.355591,-93.889359,0.0,0.0
3,1961.329956,1952-01-09 03:00:00,0,100000.000000,-2147483648,-2147483648,127.421310,61.506546,0.0,0.0
4,1961.329956,1952-03-10 03:00:00,0,100000.000000,-2147483648,-2147483648,64.063766,49.964790,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
38149301,101480.000000,2019-12-28 23:31:28,140,-1.456231,-2147483648,-2147483648,3.033260,3.319971,NaN,NaN
38149302,101500.000000,2019-12-29 11:30:27,140,-3.020738,-2147483648,-2147483648,1.961537,1.911172,NaN,NaN
38149303,101510.000000,2019-12-29 23:59:18,140,-0.516419,-2147483648,-2147483648,0.239545,0.825527,NaN,NaN
38149304,101260.000000,2019-12-30 11:32:22,140,1.692912,-2147483648,-2147483648,0.381618,0.864270,NaN,NaN


In [11]:
b = a[a.conversion_method > 0]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b

0           34
2894456     38
2901205     39
3020402    104
3118194    105
Name: observed_variable, dtype: int64


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,43000.0,1949-02-08 04:00:00,34,10.120000,0,3,NaN,NaN,NaN,NaN
1,50000.0,1949-02-08 04:00:00,34,7.340000,0,3,NaN,NaN,NaN,NaN
2,52300.0,1949-02-08 04:00:00,34,7.350000,0,3,NaN,NaN,NaN,NaN
3,59200.0,1949-02-08 04:00:00,34,15.380000,0,3,NaN,NaN,NaN,NaN
4,84400.0,1949-02-08 04:00:00,34,20.010000,0,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3215981,101480.0,2019-12-28 23:31:28,105,-1.058014,0,2,NaN,NaN,NaN,NaN
3215982,101500.0,2019-12-29 11:30:27,105,-3.730301,0,2,NaN,NaN,NaN,NaN
3215983,101510.0,2019-12-29 23:59:18,105,-0.971242,0,2,NaN,NaN,NaN,NaN
3215984,101260.0,2019-12-30 11:32:22,105,-0.862582,0,2,NaN,NaN,NaN,NaN


In [12]:
b = a[a.conversion_method > 0]
b = b[np.isnan(b['fg_depar@body'])]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b

0          34
18702      38
25451      39
46181     104
143973    105
Name: observed_variable, dtype: int64


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,43000.0,1949-02-08 04:00:00,34,10.120000,0,3,NaN,NaN,NaN,NaN
1,50000.0,1949-02-08 04:00:00,34,7.340000,0,3,NaN,NaN,NaN,NaN
2,52300.0,1949-02-08 04:00:00,34,7.350000,0,3,NaN,NaN,NaN,NaN
3,59200.0,1949-02-08 04:00:00,34,15.380000,0,3,NaN,NaN,NaN,NaN
4,84400.0,1949-02-08 04:00:00,34,20.010000,0,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
241760,101480.0,2019-12-28 23:31:28,105,-1.058014,0,2,NaN,NaN,NaN,NaN
241761,101500.0,2019-12-29 11:30:27,105,-3.730301,0,2,NaN,NaN,NaN,NaN
241762,101510.0,2019-12-29 23:59:18,105,-0.971242,0,2,NaN,NaN,NaN,NaN
241763,101260.0,2019-12-30 11:32:22,105,-0.862582,0,2,NaN,NaN,NaN,NaN


In [13]:
b = a[a.conversion_method > 0]
b = b[~np.isnan(b['fg_depar@body'])]
b = b.reset_index(drop=True)
print(b.observed_variable.drop_duplicates())
b

0          34
2875754    39
Name: observed_variable, dtype: int64


,z_coordinate,date_time,observed_variable,observation_value,conversion_flag,conversion_method,fg_depar@body,an_depar@body,biascorr@body,biascorr_fg@body
0,70000.0,1951-12-01 03:00:00,34,18.340000,0,3,-18.579998,-18.180000,NaN,NaN
1,80700.0,1951-12-01 03:00:00,34,18.000000,0,3,-10.150000,-9.190001,NaN,NaN
2,82800.0,1951-12-01 03:00:00,34,4.570000,0,3,-19.450001,-19.290001,NaN,NaN
3,85000.0,1951-12-01 03:00:00,34,3.560000,0,3,-16.020000,-16.390001,NaN,NaN
4,87000.0,1951-12-01 03:00:00,34,2.960000,0,3,-12.710000,-12.850000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2974216,90900.0,1977-12-14 12:00:00,39,0.004118,0,2,0.000417,0.000300,NaN,NaN
2974217,99500.0,1977-12-14 12:00:00,39,0.003708,0,2,-0.000348,-0.000341,NaN,NaN
2974218,100000.0,1977-12-14 12:00:00,39,0.003743,0,2,-0.000317,-0.000302,NaN,NaN
2974219,101000.0,1977-12-14 12:00:00,39,0.003706,0,2,-0.000403,-0.000360,NaN,NaN
